In [1]:
import altair as alt
import pandas as pd
from pathlib import Path

In [2]:
# %load const
CO_STO_AGOS_NL = 33087679
CO_STO_AGO_INST = 33176825
CO_STO_AGO = 33020566
CO_SAO_BENTO = 33062633
CO_SAO_VICENTE = 33063648
CO_PARQUE = 33065837
CO_ELEVA = 33178860
CO_ST_INACIO = 33063729

CO_MUN = 3304557  # Rio de Janeiro
CO_UF_RIO = 33  # RJ

MIN_ALUNOS = 30  # Mínimo de alunos fazendo Enem para considerar na análise
NUM_MELHORES = 60
ANO_ULT = 2019  # último ano a processar

In [3]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Lato');
</style>

In [4]:
def urban_theme():
    markColor = "#1696d2"
    axisColor = "#000000"
    backgroundColor = "#FFFFFF"
    font = "Lato"
    labelFont = "Lato"
    sourceFont = "Lato"
    gridColor = "#DEDDDD"
    main_palette = ["#1696d2", 
                    "#d2d2d2",
                    "#000000", 
                    "#fdbf11", 
                    "#ec008b", 
                    "#55b748", 
                    "#5c5859", 
                    "#db2b27", 
                   ]
    sequential_palette = ["#cfe8f3", 
                          "#a2d4ec", 
                          "#73bfe2", 
                          "#46abdb", 
                          "#1696d2", 
                          "#12719e", 
                         ]
    return {
        "width": 685,
        "height": 380,   
#        "autosize": "fit",
        "config": {
            "title": {
                "anchor": "start",
                "fontSize": 18,
                "font": font,
                "fontColor": "#000000"
            },
            "axisX": {
               "domain": True,
               "domainColor": axisColor,
               "domainWidth": 1,
               "grid": False,
               "labelFontSize": 12,
               "labelFont": labelFont,
               "labelAngle": 0,
               "tickColor": axisColor,
               "tickSize": 5,
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "title": "",
           },
           "axisY": {
               "domain": False,
               "grid": True,
               "gridColor": gridColor,
               "gridWidth": 1,
               "labelFontSize": 12,
               "labelFont": labelFont,
               "labelPadding": 8,
               "ticks": False,
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "titleAngle": 0,
               "titleY": -10,
               "titleX": 18,
           },
           "background": backgroundColor,
           "legend": {
               "labelFontSize": 12,
               "labelFont": labelFont,
               "symbolSize": 100,
               "symbolType": "square",
               "titleFontSize": 12,
               "titlePadding": 10,
               "titleFont": font,
               "title": "",
               "orient": "top-left",
               "offset": 0,
           },
           "view": {
               "stroke": "transparent",
           },
           "range": {
               "category": main_palette,
               "diverging": sequential_palette,
           },
           "area": {
               "fill": markColor,
           },
           "line": {
               "color": markColor,
               "stroke": markColor,
               "strokewidth": 5,
           },
           "trail": {
               "color": markColor,
               "stroke": markColor,
               "strokeWidth": 0,
               "size": 1,
           },
           "path": {
               "stroke": markColor,
               "strokeWidth": 0.5,
           },
           "point": {
               "filled": True,
           },
           "text": {
               "font": sourceFont,
               "color": markColor,
               "fontSize": 11,
               "align": "right",
               "fontWeight": 400,
               "size": 11,
           }, 
           "bar": {
                "size": 40,
                "binSpacing": 1,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": markColor,
                "stroke": False,
            }, 
       },
    }
    
    
import altair as alt
alt.themes.register("my_custom_theme", urban_theme)
alt.themes.enable("my_custom_theme")

ThemeRegistry.enable('my_custom_theme')

In [5]:
COLABORATORY = True

if COLABORATORY and 'dados_dir' not in vars(): 
  from google.colab import drive
  drive.mount('/drive')

if COLABORATORY:
  dados_dir = Path('/drive/MyDrive/Colab Notebooks/educacao/')
else:
  dados_dir = Path('./dados')

Mounted at /drive


## Colégio a processar
Mudar abaixo para ver outros colégios

In [6]:
CO_PROCESSAR = CO_PARQUE

In [7]:
df_turmas_rio = pd.read_feather(dados_dir / "turmas_rio.feather")

In [8]:
df_turmas_rio.ano.max()

2019

In [9]:
em_etapas = tuple(range(25, 39))  # são vicente deveria ter só 25-27
df_medio = df_turmas_rio[df_turmas_rio.id_etapa.isin(em_etapas) & (df_turmas_rio.id_escola == CO_SAO_VICENTE)]


In [10]:
df_medio_por_ano = df_medio.groupby("ano").agg(
    total=("num_matriculas", "sum"), no_turmas=("index", "count")
)
(
    (
        alt.Chart(df_medio_por_ano.reset_index())
        .mark_line()
        .encode(
            y=alt.Y(
                "total",
                title=None,
                scale=alt.Scale(
                    domain=[
                        df_medio_por_ano.total.min()
                        - (df_medio_por_ano.total.max() - df_medio_por_ano.total.min())
                        / 2,
                        df_medio_por_ano.total.max(),
                    ]
                ),
            ),
            x=alt.X(
                "ano:O",
                title=None,
                axis=alt.Axis(labelAngle=0, domain=False),
            ),
        )
        .properties(
            width=500,
        )
    )
    + alt.Chart(df_medio_por_ano.mean().to_frame().T)
    .mark_line(color="red")
    .encode(y="total")
    + alt.Chart(df_medio_por_ano.mean().to_frame().T)
    .mark_rule(color="orange")
    .encode(y="total")
)

alt.LayerChart(...)

## Perguntas
1. como tem variado o número de alunos do primeiro para o terceiro ano

In [11]:
df_medio_sv = df_medio[["ano", "id_etapa", "num_matriculas"]].groupby(["ano", "id_etapa"]).aggregate(
    num_matriculas=("num_matriculas", sum), num_turmas=("num_matriculas", "count")
).rename({25:'1º', 26: '2º', 27: '3º'}).reset_index().rename(columns={'id_etapa': 'Série'})#.unstack()['num_matriculas'].rename(columns={25:'1º ano', 26: '2º ano', 27: '3º ano'}).reset_index()
df_medio_sv.head()

,ano,Série,num_matriculas,num_turmas
0,2007,1º,134,4
1,2007,2º,87,2
2,2007,3º,136,4
3,2008,1º,143,4
4,2008,2º,107,3


In [12]:
alt.Chart(df_medio_sv).mark_line().encode(x='ano', y='num_matriculas', color='Série', tooltip=('num_matriculas', 'num_turmas'))

alt.Chart(...)

In [13]:
df_sv_periodo = df_medio_sv.pivot(
    index="ano", columns="Série", values="num_matriculas"
).reset_index()
df_sv_periodo["2º"] = df_sv_periodo["2º"].shift(-1)
df_sv_periodo["2º %"] = (df_sv_periodo["2º"]/df_sv_periodo["1º"] * 100).round(0)
df_sv_periodo["3º"] = df_sv_periodo["3º"].shift(-2)
df_sv_periodo["3º %"] = (df_sv_periodo["3º"]/df_sv_periodo["1º"] * 100).round(0)
df_sv_periodo["1º %"] = 100
df_sv_periodo["Período"] = (
    df_sv_periodo["ano"].astype(str) + "–" + (df_sv_periodo["ano"] + 2).astype(str)
)
df_sv_periodo_no = (
    df_sv_periodo.drop("ano", axis=1)
    .melt(id_vars=["Período"], value_vars=["1º", "2º", "3º"], var_name="Série", value_name='Nº alunos')
    .sort_values("Período")
)
df_sv_periodo_per = (
    df_sv_periodo.drop("ano", axis=1)
    .melt(id_vars=["Período"], value_vars=["1º %", "2º %", "3º %"], var_name="Série", value_name='Nº alunos')
    .sort_values("Período")
)

df_sv_periodo_per.head()

,Período,Série,Nº alunos
0,2007–2009,1º %,100.0
13,2007–2009,2º %,80.0
26,2007–2009,3º %,73.0
1,2008–2010,1º %,100.0
14,2008–2010,2º %,89.0


In [14]:
def line_chart(df):
    return (
        alt.Chart(df)
        .mark_line(interpolate="monotone", point=True)
        .encode(
            x="Série",
            y=alt.Y("Nº alunos"),#, scale=alt.Scale(zero=False)),
            color="Período:O",
            tooltip=('Período', "Nº alunos")
        )
        .properties(width=800)
    )


line_chart(df_sv_periodo_no) & line_chart(df_sv_periodo_per)

alt.VConcatChart(...)

2. como tem variado as notas dos alunos no enem (a princípio seriam comparáveis)

In [15]:
df_enem = pd.read_feather(dados_dir / "enem_anos.feather")

In [16]:
df_enem.head()

,CO_ESCOLA,Ano,nota_ciências,nota_humanas,nota_português,nota_matemática,nota_redação,nota_final,num_alunos,nota_ciências_perc80,nota_humanas_perc80,nota_português_perc80,nota_matemática_perc80,nota_redação_perc80,nota_final_perc80,rank
0,33135827,2019,647.5,651.0,627.0,789.0,940.0,758.0,31.0,676.0,685.5,641.5,823.0,960.0,778.0,1
1,33062633,2019,636.5,672.0,626.0,789.5,900.0,751.0,45.0,662.5,714.0,641.5,853.0,940.0,773.5,2
2,33178860,2019,610.0,662.5,626.0,779.0,910.0,745.5,60.0,652.5,683.5,648.0,848.0,940.0,779.0,3
3,33135371,2019,613.0,647.0,620.5,744.0,940.0,743.0,99.0,649.5,681.0,641.0,825.0,960.0,769.0,4
4,33176825,2019,605.5,649.0,620.5,759.0,920.0,742.0,146.0,643.0,682.0,644.5,815.5,960.0,769.0,5


In [17]:
alt.Chart(
    df_enem[df_enem.CO_ESCOLA.isin([CO_SAO_VICENTE])]
    # df_enem.query("rank <= 30 and NU_ANO == @ANO_ULT").CO_ESCOLA)]
).mark_line(point=True).encode(
    x="Ano:O",
    y=alt.Y("nota_matemática"),
    color="CO_ESCOLA:N",
    tooltip=("rank", "nota_final", "nota_final_perc80", 'nota_matemática'),
).properties(
    width=800
)

alt.Chart(...)

In [18]:
materias = alt.Chart(
    df_enem[df_enem.CO_ESCOLA == CO_SAO_VICENTE].melt(
        id_vars="Ano",
        value_vars=[c for c in df_enem.columns if c.startswith("nota_") and not c.endswith('80')],
        var_name="Matéria",
        value_name="Nota",
    )
).mark_line(point=True).encode(
    x="Ano:O",
    y=alt.Y("Nota", scale=alt.Scale(zero=False)),
    color="Matéria", tooltip=[ "Matéria", "Nota","Ano"]
    #tooltip=(f'{i}:Q' for i in df_enem.columns if i.startswith('nota_')),
).properties(
    width=800
)
materias

alt.Chart(...)

In [19]:
df_enem[:5]#.head()

,CO_ESCOLA,Ano,nota_ciências,nota_humanas,nota_português,nota_matemática,nota_redação,nota_final,num_alunos,nota_ciências_perc80,nota_humanas_perc80,nota_português_perc80,nota_matemática_perc80,nota_redação_perc80,nota_final_perc80,rank
0,33135827,2019,647.5,651.0,627.0,789.0,940.0,758.0,31.0,676.0,685.5,641.5,823.0,960.0,778.0,1
1,33062633,2019,636.5,672.0,626.0,789.5,900.0,751.0,45.0,662.5,714.0,641.5,853.0,940.0,773.5,2
2,33178860,2019,610.0,662.5,626.0,779.0,910.0,745.5,60.0,652.5,683.5,648.0,848.0,940.0,779.0,3
3,33135371,2019,613.0,647.0,620.5,744.0,940.0,743.0,99.0,649.5,681.0,641.0,825.0,960.0,769.0,4
4,33176825,2019,605.5,649.0,620.5,759.0,920.0,742.0,146.0,643.0,682.0,644.5,815.5,960.0,769.0,5


In [20]:
df_enem[df_enem.CO_ESCOLA == CO_SAO_VICENTE].melt(
        id_vars=["Ano", 'CO_ESCOLA'],
        value_vars=[c for c in df_enem.columns if c.startswith("nota_")],
        var_name="Matéria",
        value_name="Nota",
    )

,Ano,CO_ESCOLA,Matéria,Nota
0,2019,33063648,nota_ciências,589.0
1,2018,33063648,nota_ciências,603.5
2,2017,33063648,nota_ciências,622.5
3,2016,33063648,nota_ciências,632.5
4,2015,33063648,nota_ciências,632.0
...,...,...,...,...
79,2017,33063648,nota_final_perc80,747.0
80,2016,33063648,nota_final_perc80,745.5
81,2015,33063648,nota_final_perc80,753.5
82,2014,33063648,nota_final_perc80,730.0


3. em que percentil tem ficado as notas?

In [21]:
df_turmas_rio.head()

,index,ano,id_escola,id_etapa,id_municipio,id_uf,num_matriculas
0,6791,2007,33056480,21.0,3303302,33,32
1,6798,2007,33056480,21.0,3303302,33,34
2,6802,2007,33018278,2.0,3306008,33,23
3,6808,2007,33056480,21.0,3303302,33,32
4,6814,2007,33056480,21.0,3303302,33,31


In [22]:
df_medio_alunos = (df_turmas_rio.loc[df_turmas_rio.id_etapa.isin(em_etapas), 
                                     ["ano", "id_escola", "id_etapa", "num_matriculas"]]
                   .groupby([ "id_escola", "ano", "id_etapa"])
                   .aggregate(num_matriculas=("num_matriculas", sum), num_turmas=("num_matriculas", "count"))
                   .rename({25:'1º', 26: '2º', 27: '3º'})
                   .reset_index()
                   .rename(columns={'id_etapa': 'Série'})
                   )#.unstack()['num_matriculas'].rename(columns={25:'1º ano', 26: '2º ano', 27: '3º ano'}).reset_index()
df_medio_alunos.head()

,id_escola,ano,Série,num_matriculas,num_turmas
0,33000026,2007,1º,90.0,3
1,33000026,2007,2º,51.0,2
2,33000026,2007,3º,60.0,3
3,33000026,2008,1º,73.0,3
4,33000026,2008,2º,48.0,2


In [23]:
df_medio = df_turmas_rio[df_turmas_rio.id_etapa.isin(em_etapas)]
df_sv_periodo = df_medio_sv.pivot(
    index="ano", columns="Série", values="num_matriculas"
).reset_index()
df_sv_periodo["2º"] = df_sv_periodo["2º"].shift(-1)
df_sv_periodo["2º %"] = (df_sv_periodo["2º"]/df_sv_periodo["1º"] * 100).round(0)
df_sv_periodo["3º"] = df_sv_periodo["3º"].shift(-2)
df_sv_periodo["3º %"] = (df_sv_periodo["3º"]/df_sv_periodo["1º"] * 100).round(0)
df_sv_periodo["1º %"] = 100
df_sv_periodo["Período"] = (
    df_sv_periodo["ano"].astype(str) + "–" + (df_sv_periodo["ano"] + 2).astype(str)
)
df_sv_periodo_no = (
    df_sv_periodo.drop("ano", axis=1)
    .melt(id_vars=["Período"], value_vars=["1º", "2º", "3º"], var_name="Série", value_name='Nº alunos')
    .sort_values("Período")
)
df_sv_periodo_per = (
    df_sv_periodo.drop("ano", axis=1)
    .melt(id_vars=["Período"], value_vars=["1º %", "2º %", "3º %"], var_name="Série", value_name='Nº alunos')
    .sort_values("Período")
)

df_sv_periodo_per.head()

,Período,Série,Nº alunos
0,2007–2009,1º %,100.0
13,2007–2009,2º %,80.0
26,2007–2009,3º %,73.0
1,2008–2010,1º %,100.0
14,2008–2010,2º %,89.0


In [24]:
num_escolas = 60

In [26]:
df_enem2 = df_enem[:num_escolas].rename(columns={'CO_ESCOLA': 'id_escola'})
df_notas = df_enem.loc[df_enem.CO_ESCOLA.isin(df_enem2.id_escola), :].rename(columns={'CO_ESCOLA': 'id_escola'})
df_notas_melt= (df_notas.set_index(['id_escola', 'Ano' ]) # primeiro cria chave com null in valores inexistentes
                .reindex(pd.MultiIndex.from_product([df_notas.id_escola.unique(), df_notas.Ano.unique()], names=['id_escola', 'Ano']))
                .reset_index()
                .melt(
                  id_vars=["Ano", 'id_escola'],
                  value_vars=[c for c in df_enem.columns if c.startswith("nota_") and not c.endswith('80')],
                  var_name="Matéria",
                  value_name="Nota",
              ))
df_notas_melt.head()

,Ano,id_escola,Matéria,Nota
0,2019,33135827,nota_ciências,647.5
1,2018,33135827,nota_ciências,650.5
2,2017,33135827,nota_ciências,668.0
3,2016,33135827,nota_ciências,685.5
4,2015,33135827,nota_ciências,682.0


In [115]:
num_anos = 5
df_medio_alunos2 = df_medio_alunos[df_medio_alunos.id_escola.isin(df_enem2.id_escola) & (df_medio_alunos.ano > (df_medio_alunos.ano.max() - num_anos))].copy()
df_medio_alunos2["Período"] = (
    df_medio_alunos2["ano"].astype(str) + "–" + (df_medio_alunos2["ano"] + 2).astype(str)
)
alunos_index_names = ['id_escola', 'Período', 'Série']
new_alunos2_index = pd.MultiIndex.from_product([df_medio_alunos2.id_escola.unique(), 
                                                df_medio_alunos2.Período.unique(), 
                                                ['1º', '2º', '3º']], 
                                               names=alunos_index_names)

df_medio_alunos2 = df_medio_alunos2.set_index(alunos_index_names).reindex(new_alunos2_index).reset_index()



In [29]:
df_notas.shape, df_notas_melt.shape, df_enem2.shape, 

((352, 16), (2520, 4), (60, 16))

In [65]:
def enem_ajusta_cols(df):
    return df.rename(
        columns={
            "NU_ANO": "Ano",
            "NU_NOTA_CN": "nota_ciências",
            "NU_NOTA_CH": "nota_humanas",
            "NU_NOTA_LC": "nota_português",
            "NU_NOTA_MT": "nota_matemática",
            "NU_NOTA_REDACAO": "nota_redação",
            'CO_ESCOLA': 'id_escola'
        }
    )


In [67]:
df_enem_bins = enem_ajusta_cols(pd.read_feather(dados_dir / 'enem_rio_bins.feather'))
(df_enem_bins).head()

,id_escola,nota_ciências,nota_humanas,nota_português,nota_matemática,nota_redação,nota_final,bins,bin_label
0,33000026,0.0,0,0,0.0,1,0.0,"(-25.001, 25.0]",0
1,33000026,3.0,0,0,0.0,0,0.0,"(25.0, 75.0]",50
2,33000026,1.0,1,0,4.0,0,0.0,"(75.0, 125.0]",100
3,33000026,2.0,3,3,1.0,0,1.0,"(125.0, 175.0]",150
4,33000026,2.0,4,1,2.0,1,0.0,"(175.0, 225.0]",200


In [86]:
df_enem_bins2 = df_enem_bins.loc[df_enem_bins.id_escola.isin(df_enem2.id_escola), :]
df_enem_bins2.columns

Index(['id_escola', 'nota_ciências', 'nota_humanas', 'nota_português',
       'nota_matemática', 'nota_redação', 'nota_final', 'bins', 'bin_label'],
      dtype='object')

In [58]:
brush = alt.selection_single(fields=['id_escola'], 
                             nearest=True, 
                             init={'id_escola': CO_SAO_VICENTE})

In [116]:

base = alt.Chart(df_enem2, title=alt.TitleParams("Medianas da nota final (clique para selecionar uma escola)", subtitle=["Redação tem peso 2", "Área cinza mostra o 80º percentil"]))

median = (base.mark_bar(size=30, color='cornflowerblue')
          .encode(x=alt.X("id_escola:N", sort=alt.SortField("rank"), axis=alt.Axis(labelFontSize=10)), 
                  y=alt.Y("nota_final:Q", axis=alt.Axis(title=None)), 
                  tooltip=['nota_final', 'nota_final_perc80', 'rank', 'id_escola'],
                  opacity=alt.condition(brush, alt.value(1), alt.value(.6)))
 
).add_selection(brush)

perc80 = (base.mark_bar(size=30,  color='lightgray', opacity=.8)
          .encode(x=alt.X("id_escola:N", sort=alt.SortField("rank")), 
                  y="nota_final_perc80:Q")
 
)
#alt.data_transformers.enable(max_rows=None)

materias = (alt.Chart(df_notas_melt)
           .mark_line(point=True)
           .encode(
                x=alt.X("Ano:O", scale=alt.Scale(padding=0.05)),
                y=alt.Y("Nota", scale=alt.Scale(domain=[500, 1000]), axis=alt.Axis(title=None)),
                color=alt.Color("Matéria",legend=alt.Legend(orient='top'), scale=alt.Scale(scheme="tableau10"), 
                                sort=['nota_final', 'nota_redação', 'nota_matemática', 'nota_português', 'nota_ciências','nota_humanas']),
                tooltip=["Matéria", "Nota","Ano"]
                )
            #.transform_filter(alt.FieldEqualPredicate(field="id_escola", equal=CO_SAO_BENTO))
            .transform_filter(brush)
            .properties(
                width=500, title="Notas a cada ano"
            )
           )

nalunos = (alt.Chart(df_medio_alunos2)
           .mark_line(point=True)
           .encode(
              x=alt.X("Série:O", scale=alt.Scale(domain=['1º', '2º', '3º'], padding=0.05)),
              y=alt.Y("num_matriculas:Q", axis=alt.Axis(title=None), scale=alt.Scale(domain=[0,230])),
              color=alt.Color("Período:O", scale=alt.Scale(scheme='blues'), legend=alt.Legend(orient='top')), 
              tooltip=('Período', "Série", "num_matriculas")
            ).transform_filter(brush)
         ).properties(width=400, title="Nº matrículas por série")

notas_dist_width=400
notas_dist = (
    alt.Chart(df_enem_bins2)
       .mark_bar(stroke='white', width=notas_dist_width//21)
       .encode(x="bin_label:O", 
               y='nota_final:Q',
               tooltip=['nota_final', 'bins'])
      .transform_filter(brush)
      .properties(width=notas_dist_width, title="Histograma das notas")
)         

alt.vconcat( 
    (perc80 + median).properties(width=alt.Step(45)).interactive(),
    (materias.interactive() | nalunos.interactive() | notas_dist.interactive()).resolve_scale(color='independent')
).configure_point(size=40)

alt.VConcatChart(...)

In [31]:
df_medio_alunos2.head()

,id_escola,Período,Série,ano,num_matriculas,num_turmas
0,33021724,2014–2016,1º,2014.0,59.0,2.0
1,33021724,2014–2016,2º,2014.0,65.0,2.0
2,33021724,2014–2016,3º,2014.0,37.0,1.0
3,33021724,2015–2017,1º,2015.0,64.0,2.0
4,33021724,2015–2017,2º,2015.0,53.0,2.0


In [32]:
df_notas_melt

,Ano,id_escola,Matéria,Nota
0,2019,33135827,nota_ciências,647.5
1,2018,33135827,nota_ciências,650.5
2,2017,33135827,nota_ciências,668.0
3,2016,33135827,nota_ciências,685.5
4,2015,33135827,nota_ciências,682.0
...,...,...,...,...
2515,2017,33041105,nota_final,671.0
2516,2016,33041105,nota_final,647.0
2517,2015,33041105,nota_final,666.5
2518,2014,33041105,nota_final,642.0
